<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#读取-HWDB-数据" data-toc-modified-id="读取-HWDB-数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>读取 HWDB 数据</a></span><ul class="toc-item"><li><span><a href="#真实训练数据" data-toc-modified-id="真实训练数据-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>真实训练数据</a></span></li><li><span><a href="#测试数据-MNIST" data-toc-modified-id="测试数据-MNIST-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>测试数据 MNIST</a></span><ul class="toc-item"><li><span><a href="#将-gz-格式的-mnist-数据转换为-png-格式" data-toc-modified-id="将-gz-格式的-mnist-数据转换为-png-格式-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>将 gz 格式的 mnist 数据转换为 png 格式</a></span></li></ul></li></ul></li><li><span><a href="#数据预处理" data-toc-modified-id="数据预处理-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>数据预处理</a></span></li><li><span><a href="#模型定义与初始化" data-toc-modified-id="模型定义与初始化-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>模型定义与初始化</a></span><ul class="toc-item"><li><span><a href="#SigleDense_Custom" data-toc-modified-id="SigleDense_Custom-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>SigleDense_Custom</a></span></li><li><span><a href="#ResNet_Custom" data-toc-modified-id="ResNet_Custom-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>ResNet_Custom</a></span></li><li><span><a href="#Xception_Keras" data-toc-modified-id="Xception_Keras-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Xception_Keras</a></span></li></ul></li><li><span><a href="#训练" data-toc-modified-id="训练-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>训练</a></span><ul class="toc-item"><li><span><a href="#sigle_dense_v.01" data-toc-modified-id="sigle_dense_v.01-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>sigle_dense_v.01</a></span></li><li><span><a href="#sigle_dense_v.02" data-toc-modified-id="sigle_dense_v.02-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>sigle_dense_v.02</a></span></li><li><span><a href="#ResNet_v0.1" data-toc-modified-id="ResNet_v0.1-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>ResNet_v0.1</a></span></li></ul></li></ul></div>

## Setup

In [42]:
import os
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input
from keras.optimizers import Nadam, Adam, SGD
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.models import Model

batch_size = 512

## 读取 HWDB 数据

### 真实训练数据

In [1]:
train_dir = '/aiml/data/train/'
test_dir = '/aiml/data/test/'
log_dir = '/aiml/dfs/checkpoint/train/'
pix = 64

### 测试数据 MNIST

主要用于确定数据输入正确

In [18]:
train_dir = '/aiml/data/mnist_train'
test_dir = '/aiml/data/mnist_test'
log_dir = '/aiml/dfs/checkpoint/train/'
pix = 28

#### 将 gz 格式的 mnist 数据转换为 png 格式

下载并解压 mnist 数据集

In [3]:
# 获取脚本 https://github.com/myleott/mnist_png
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!zcat train-images-idx3-ubyte.gz > train-images-idx3-ubyte
!zcat train-labels-idx1-ubyte.gz > train-labels-idx1-ubyte
!pip install pypng

--2018-01-06 13:57:29--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving proxyhk.huawei.com (proxyhk.huawei.com)... 172.18.32.221
Connecting to proxyhk.huawei.com (proxyhk.huawei.com)|172.18.32.221|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K  --.-KB/s    in 0.04s   

2018-01-06 13:57:30 (697 KB/s) - ‘train-labels-idx1-ubyte.gz’ saved [28881/28881]

--2018-01-06 13:57:30--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving proxyhk.huawei.com (proxyhk.huawei.com)... 172.18.32.221
Connecting to proxyhk.huawei.com (proxyhk.huawei.com)|172.18.32.221|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

     train-images-i  61%[===========>        ]   5.77M   552KB/s    eta 8s     ^C

gzip: trai

解压有问题，所以需要将训练集分成两部分

In [ ]:
for num in [str(x) for x in range(10)]:
    train_dir = os.path.join('training', num)
    test_dir = os.path.join('testing', num)
    train_files = [os.path.join(train_dir, x) for x in os.listdir(train_dir)]
    test_files = [os.path.join(test_dir, x) for x in os.listdir(test_dir)]
    train_files.sort()
    test_files.sort()
    [os.remove(x) for x in train_files[:500]]
    [os.remove(x) for x in test_files[500:]]
!find . -name "*.png"|cut -d '/' -f 4 |sort |uniq |wc -l
!find . -name "*.png"| wc -l

## 数据预处理

In [4]:
train_data = image.ImageDataGenerator(samplewise_std_normalization=True)
test_data = image.ImageDataGenerator(samplewise_std_normalization=True)
train_gen = train_data.flow_from_directory(train_dir, color_mode='grayscale', target_size=(pix, pix), batch_size=batch_size)
test_gen = train_data.flow_from_directory(test_dir, color_mode='grayscale', target_size=(pix, pix), batch_size=batch_size)
# 类别数量
num_class = train_gen.num_classes
# 训练样本数量
num_train = train_gen.n
# 测试样本数量
num_test = test_gen.n
# 确认读取的图片的格式正确
image_shape = train_gen.next()[0].shape
# 查看返回的标签类型，“categorical” 表示为 one-hot 标签
print(train_gen.class_mode)
print(num_class, num_train, num_test, image_shape)

Found 55000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
categorical
10 55000 5000 (512, 28, 28, 1)


## 模型定义与初始化

### SigleDense_Custom

主要用于测试数据输入正确性，发现loss俨然有上升趋势，放弃其他尝试，换上 MNIST 后，发现一切正常，说明 HCCR 需要的模型较为复杂，收敛时间更长

In [27]:
model_dir_name = 'sigle_dense'

def sigle_dense_model():
    model = Sequential([
        Flatten(input_shape=(pix, pix, 1)),
        Dense(512, activation='softmax'),
        Dense(num_class, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = sigle_dense_model()

### ResNet_Custom

In [47]:
from __future__ import division
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model
import six

def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3

def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier

def _bn_relu(input):
    """
    Helper to build a BN -> relu block
    """

    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)

def _conv_bn_relu(**conv_params):

    """
    Helper to build a conv -> BN -> relu block
    """

    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,strides=strides, padding=padding,kernel_initializer=kernel_initializer,kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)
    return f

def _bn_relu_conv(**conv_params):

    """
    Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """

    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,strides=strides, padding=padding, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)(activation)
    return f

def _shortcut(input, residual):

    """
    Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.

    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input

    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS], kernel_size=(1, 1), strides=(stride_width, stride_height), padding="valid", kernel_initializer="he_normal",kernel_regularizer=l2(0.0001))(input)
    return add([shortcut, residual])

def _residual_block(block_function, filters, repetitions, is_first_layer=False):

    """
    Builds a residual block with repeating bottleneck blocks.
    """

    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides, is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input
    return f

def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """
    Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """

    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),strides=init_strides, padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3), strides=init_strides)(input)
        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)
    return f

def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """
    Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """

    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1), strides=init_strides, padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1), strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)
    return f

class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """
        Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """

        _handle_dim_ordering()

        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[0], input_shape[1], input_shape[2])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]), strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal", activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model


    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])

In [49]:
model_dir_name = 'Resnet18'
model = ResnetBuilder.build_resnet_18((pix, pix, 1), num_class)
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

### Xception_Keras

2017年的state-of-the-art图像分类模型，优先尝试，[官方说明](https://keras-cn.readthedocs.io/en/latest/other/application/#xception)
从函数说明中可以看出，input_shape如果要修改就必须设置 include_top=False，也就说删除顶层的全连接层，但是如果要设置 classes 又需要这些全连接层，花了不少功夫找到了[解决方法](https://github.com/keras-team/keras/issues/4465)，然而，通道数被限制为3，放弃

In [46]:
model_dir_name = 'xception_keras'

def get_xception_keras_model():
    model_xception = Xception(weights=None, include_top=False)
    input_layer = Input(shape=(64, 64, 1), name='image_input')
    output_model_xception = model_xception(input_layer)
    x = Flatten(name='flatten')(output_model_xception)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(num_class, activation='softmax', name='predictions')(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

model = get_xception_keras_model()

ValueError: number of input channels does not match corresponding dimension of filter, 1 != 3

## 训练

### sigle_dense_v.01

主要用于测试 ModelCheckpoint/TensorBoard 是否生效，测试发现不生效，尝试将数据保存至train下

In [29]:
# 定义存储目录
# train_dir_name = os.path.join(log_dir, model_dir_name, 'v.01')
train_dir_name = os.path.join(log_dir, 'train')

# 通用代码，无需更改
if os.path.exists(train_dir_name):
    os.system('rm -rf %s' % (train_dir_name));
print(train_dir_name)
check_cb = ModelCheckpoint(filepath=os.path.join(train_dir_name , 'weights.hdf5'), verbose=1, save_best_only=True)
board_cb = TensorBoard(log_dir=train_dir_name, histogram_freq=False, write_graph=True, write_images=False)

# 开始训练
model.fit_generator(train_gen, steps_per_epoch=10, epochs=3, 
                    validation_data=test_gen, validation_steps=1, 
                    callbacks=[check_cb, board_cb])

0

/aiml/dfs/checkpoint/train
Epoch 1/3
10/10 [==============================] - 10s 961ms/step - loss: 2.1703 - acc: 0.8099 - val_loss: 2.1540 - val_acc: 0.7988
Epoch 2/3
10/10 [==============================] - 7s 675ms/step - loss: 2.1422 - acc: 0.8170 - val_loss: 2.1271 - val_acc: 0.8340
Epoch 3/3
10/10 [==============================] - 7s 698ms/step - loss: 2.1127 - acc: 0.8490 - val_loss: 2.0994 - val_acc: 0.8594


### sigle_dense_v.02

主要用于测试生成的权重文件是否可以使用

In [31]:
# 定义存储目录
train_dir_name = os.path.join(log_dir, model_dir_name, 'v.02')

# 通用代码，无需更改
if os.path.exists(train_dir_name):
    os.system('rm -rf %s' % (train_dir_name));
print(train_dir_name)
weights_name = os.path.join(train_dir_name , 'weights.hdf5')
check_cb = ModelCheckpoint(filepath=weights_name, verbose=1, save_best_only=True)
board_cb = TensorBoard(log_dir=train_dir_name, histogram_freq=False, write_graph=False, write_images=False)

# 开始训练
model.fit_generator(train_gen, steps_per_epoch=num_train//batch_size, epochs=4, 
                    validation_data=test_gen, validation_steps=num_test//batch_size, 
                    callbacks=[check_cb, board_cb])

/aiml/dfs/checkpoint/sigle_dense/v.02
Epoch 1/4
107/107 [==============================] - 32s 302ms/step - loss: 2.1322 - acc: 0.8162 - val_loss: 1.9962 - val_acc: 0.8843
Epoch 2/4
107/107 [==============================] - 29s 269ms/step - loss: 1.8712 - acc: 0.9061 - val_loss: 1.7649 - val_acc: 0.9110
Epoch 3/4
107/107 [==============================] - 30s 282ms/step - loss: 1.6581 - acc: 0.9268 - val_loss: 1.5782 - val_acc: 0.9206
Epoch 4/4
107/107 [==============================] - 26s 246ms/step - loss: 1.4817 - acc: 0.9332 - val_loss: 1.4200 - val_acc: 0.9191


In [33]:
weights_name

'/aiml/dfs/checkpoint/sigle_dense/v.02/weights.hdf5'

这里没有搞清楚 model.load_weights(filepath, by_name=False) 中 by_name 的作用，反正用了就预测错误

In [38]:
model = sigle_dense_model()
model.load_weights('/aiml/dfs/checkpoint/sigle_dense/v.02/weights.hdf5')
model.evaluate_generator(test_gen)
len(model.predict_generator(test_gen))

[1.4215239402770996, 0.9181999987602234]

5000

### ResNet_v0.1

优先使用 MNist 测试集，看该模型是否收敛，确认模型正确性

In [ ]:
# 定义存储目录
train_dir_name = os.path.join(log_dir, model_dir_name, 'v.01')

# 通用代码，无需更改
if os.path.exists(train_dir_name):
    os.system('rm -rf %s' % (train_dir_name));
print(train_dir_name)
weights_name = os.path.join(train_dir_name , 'weights.hdf5')
check_cb = ModelCheckpoint(filepath=weights_name, verbose=1, save_best_only=True)
board_cb = TensorBoard(log_dir=train_dir_name, histogram_freq=False, write_graph=False, write_images=False)

# 开始训练
model.fit_generator(train_gen, steps_per_epoch=num_train//batch_size, epochs=4, 
                    validation_data=test_gen, validation_steps=num_test//batch_size, 
                    callbacks=[check_cb, board_cb])

/aiml/dfs/checkpoint/Resnet18/v.01
Epoch 1/4
107/107 [==============================] - 301s 3s/step - loss: 0.6067 - acc: 0.9618 - val_loss: 0.6775 - val_acc: 0.9236
Epoch 2/4
 86/107 [=======================>......] - ETA: 55s - loss: 0.4155 - acc: 0.9886